In [1]:
from task import *

/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


path is  /data/mn27889/mental-health-agents/src/sdoh_analysis/helpers/../models/setfit-mental-health-topic-prediction/setfit-small-research-topic-prediction/content/setfit-small-research-topic-prediction
True


/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  w

encoding database


2775it [00:00, 480026.13it/s]


got 2511 of 2749 others out bc of size


100%|██████████| 883/883 [00:00<00:00, 312432.13it/s]


encodnig databsae - this can take a while - TODO - cache this
db encoded


In [2]:
import re

In [3]:
task = MentalHealthTask()

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
idx = 0
x = task.get_input(idx)

In [5]:
print(x)

Life is so stagnant. I can't seem to motivate myself to care about anything outside of what I'm told is needed to be done by me. I go to work because I'm told it's what I need to do to support my mom and sister. So I do what needs to be done, but when all is done, I feel like I'm rotting in a perfectly good body that should've gone to someone who would more enjoy it.

There is no satisfaction in my job, I work alone all day tortured by my own mind for 8 hours day, this pushes me to the point of physically beating on myself in hopes I calm down enough to keep working. I've started smoking whatever is able to be acquired legally to get theiught work. I wake up dreading work and miserable, wanting all day for it to end, just to come home and then rot doing nothing. I'll listen to music, or play games but only because it's what I used to do, I don't feel nearly any joy playing games anymore compared to how it used to be. I buy more games, to then not play them and complain there is nothing

In [6]:
determinant_prompt, example_format, sdoh_prompt = task.get_cot_sdoh_prompt_data(input=x, idx=idx)

In [7]:
final_prompt = user_prompt_persona.format(user_input=x, user_past_history=sdoh_prompt)

In [8]:
print(final_prompt)

You are given a user input shared by a mentally ill person along with the past history of the same person categorized by social determinants.
Considering the user input and the past history of the user, your job is to determine 3 therapy techniques out of the following 8 therapy techniques
to better improve the overall mental health of the person: (1) Cognitive Behavioral Therapy (2) Exposure Therapy (3) Psychodynamic Therapy (4) Client Centered Therapy
(5) Humanistic Therapy (6) Interpersonal Therapy (7) Mentalization Therapy (8) Mindfulness Therapy.

User Input: Life is so stagnant. I can't seem to motivate myself to care about anything outside of what I'm told is needed to be done by me. I go to work because I'm told it's what I need to do to support my mom and sister. So I do what needs to be done, but when all is done, I feel like I'm rotting in a perfectly good body that should've gone to someone who would more enjoy it.

There is no satisfaction in my job, I work alone all day t

In [9]:
# output = gpt(user_prompt=final_prompt, system_prompt=sys_prompt_persona)
# answer = output[0]
# print(answer)

Therapy Technique: Cognitive Behavioral Therapy | Interpersonal Therapy | Mindfulness Therapy


In [27]:
answer  = 'Therapy Technique: Cognitive Behavioral Therapy | Interpersonal Therapy | Mindfulness Therapy'

In [28]:
match = re.match(r".*Therapy Technique:([\s\|\w]+).*", answer, re.DOTALL)

In [29]:
therapy_techniques = match.groups()[0]
therapy_list = therapy_techniques.split('|')
therapy_list = [therapy.strip() for therapy in therapy_list]

In [30]:
final_prompt_therapy = user_prompt_therapy.format(therapy_technique=therapy_list[0], user_input=x, user_past_history=sdoh_prompt)

In [31]:
print(final_prompt_therapy)

You are given a user input shared by a mentally ill person along with the past history of the same person categorized by social determinants.
Considering the user input and the past history, your job is to adopt the practices used in Cognitive Behavioral Therapy and provide mental health support to this person such that it improves the overall mental health of the person.
You firstly need to make a comprehensive personalized plan and then write a comprehensive counselling advice based on that plan. You should personalize your plan and counselling advice
according to the specific scenario of this person i.e., you should mention information provided in the user input as well as the past history and then try to develop
your plan/counselling around this information.

User Input: Life is so stagnant. I can't seem to motivate myself to care about anything outside of what I'm told is needed to be done by me. I go to work because I'm told it's what I need to do to support my mom and sister. So

In [32]:
# output = gpt(user_prompt=final_prompt_therapy, temperature=0.7, n=3)

In [33]:
output

["Therapy Plan:\n\n1. Establish Trust and Safety: The primary goal is to establish a safe environment where the user feels comfortable expressing their feelings. This involves maintaining confidentiality and showing empathy towards the user's situation.\n\n2. Cognitive Restructuring: This involves identifying and challenging the user's negative thoughts and beliefs. For this person, this would involve addressing their feelings of hopelessness, worthlessness, and suicidal thoughts.\n\n3. Behavioral Activation: Help the user to engage in activities they previously enjoyed or may enjoy, to help reduce their feelings of hopelessness and increase their motivation.\n\n4. Grief Counseling: Given the user's fear of losing their mother and the impact of their father's death, grief counseling could be beneficial. This would involve helping the user process their feelings of loss and fear.\n\n5. Coping Skills and Stress Management: Teach the user effective coping strategies to handle their daily 

In [59]:
example_output =[
"Therapy Plan:\n\n1. Establish Trust and Safety: The primary goal is to establish a safe environment where the user feels comfortable expressing their feelings. This involves maintaining confidentiality and showing empathy towards the user's situation.\n\n2. Cognitive Restructuring: This involves identifying and challenging the user's negative thoughts and beliefs. For this person, this would involve addressing their feelings of hopelessness, worthlessness, and suicidal thoughts.\n\n3. Behavioral Activation: Help the user to engage in activities they previously enjoyed or may enjoy, to help reduce their feelings of hopelessness and increase their motivation.\n\n4. Grief Counseling: Given the user's fear of losing their mother and the impact of their father's death, grief counseling could be beneficial. This would involve helping the user process their feelings of loss and fear.\n\n5. Coping Skills and Stress Management: Teach the user effective coping strategies to handle their daily stressors, especially related to their job and familial responsibilities.\n\n6. Substance Use Intervention: Given the user's mention of smoking to cope with work, addressing substance use would be important. This could involve providing resources for substance abuse or exploring healthier coping mechanisms.\n\nTherapy Counsel:\n\nI appreciate your honesty in expressing your feelings, and it's important to remember that it's okay to feel this way. However, it's equally important to remember that there are healthier ways to cope with these feelings.\n\nOne thing that could be beneficial is to engage in activities that you previously enjoyed. You mentioned listening to music and playing games. Even if you don't feel the same joy, it's still important to engage in these activities, as they can help to break up the monotony of your daily routine.\n\nIt's clear that you're carrying a heavy burden, especially with the loss of your father and your fear of losing your mother. It's perfectly normal to feel this way, and it's okay to grieve. However, it's also important to remember that it's okay to seek help, whether it be from a professional or from your loved ones.\n\nYou also mentioned feeling like a black sheep and like you're bringing others down. It's important to remember that everyone has their own struggles, and it's okay to not always be happy. However, it's also important to not isolate yourself, as this can exacerbate feelings of loneliness and despair.\n\nLastly, you mentioned using substances to help cope with work. While this may provide temporary relief, it's not a healthy long-term solution. There are other, healthier ways to cope with stress, such as exercise, meditation, or talking to someone about your feelings.\n\nIt's clear that you're going through a lot, and it's okay to seek help. You don't have to go through this alone. There are resources available to help you, and it's okay to use them. You're not alone, and it's okay to ask for help.",
"Therapy Plan:\n1. Building Trust and Rapport: The first step is to establish a strong therapeutic relationship. This includes making the individual feel heard, validated, and understood. It is important to provide a safe and non-judgmental space where they can express themselves freely.\n\n2. Crisis Management: Given the severity of the individual's feelings of despair and suicidal ideation, immediate crisis intervention measures need to be taken. This could involve involving a local crisis team or ensuring they have access to emergency mental health services. \n\n3. Cognitive Behavioral Therapy (CBT): The goal of CBT is to help the individual understand how their thoughts and behaviors are related to their feelings. In this case, CBT can be used to challenge the individual's negative thought patterns, specifically around their work, their role within their family, and their overall outlook on life. \n\n4. Activity Scheduling: This technique will help the individual to structure their day and incorporate activities they previously enjoyed, such as gaming or listening to music. The intention is to slowly re-engage the individual with activities they once found pleasurable.\n\n5. Dealing with Loss and Grief: Given the significant impact of their father's death, grief counseling might be necessary. This can help them process their feelings of loss and learn to cope in a healthy manner.\n\n6. Supportive Family Therapy: Incorporating family members, specifically the mother and sister, into some therapy sessions may provide additional support and understanding for the individual's mental health struggles.\n\n7. Coping Strategies and Skills: Teach the person coping mechanisms to handle stress, anxiety, and depressive thoughts.\n\nTherapy Counseling:\nThank you for sharing your feelings and experiences with me. I can sense the pain you're feeling, and I want you to know that your emotions are valid. It's okay to feel overwhelmed and it's okay to not have all the answers. Many people feel lost and stuck in life, especially when dealing with significant life changes and losses. \n\nI also want you to know that there is help available for you. It sounds like you have a lot of responsibilities, and it's understandable that you're feeling overwhelmed. It's okay to seek help and to take care of your own mental health. In fact, it's necessary. \n\nCognitive Behavioral Therapy can be an effective way for us to address some of the thoughts and feelings you're experiencing. We can work together to identify negative thought patterns and find ways to challenge them. This may help you to see your situation in a new light and find more satisfaction in your daily life.\n\nYour love for your mom and sister is evident, and it's clear that you want to support them. It's important to remember that taking care of your own mental health is not a selfish act. On the contrary, it can allow you to be more present and supportive for your loved ones.\n\nIn terms of your job, we can explore ways to make your workday more tolerable and less isolating. This might involve finding pockets of time for breaks, integrating activities you enjoy into your workday, or seeking support from colleagues or supervisors.\n\nYou mentioned that you used to enjoy playing games and listening to music. These activities can be a source of relief and joy. We can work on incorporating these activities back into your daily routine to help you experience enjoyment again.\n\nLastly, I want to acknowledge your courage. It takes strength to share your feelings and seek help. Please remember that you're not alone in this journey. Together, we can navigate through these challenges and work towards improving your mental health.",
'Therapy Plan:\n\n1. Establishing Therapeutic Trust: Given the severity of the client\'s distress, it is crucial to establish a strong therapeutic alliance, reassuring them that they are in a safe, non-judgmental environment where they can express their feelings openly.\n\n2. Crisis Intervention: Given the client\'s suicidal ideation, immediate crisis intervention strategies should be implemented. Encourage the client to contact local mental health crisis services or a trusted individual when experiencing suicidal thoughts. \n\n3. Psychoeducation: Educate the client about depression, its symptoms, and the cognitive-behavioral model, which can help them understand their current emotional state.\n\n4. Cognitive Restructuring: This involves identifying and challenging negative thought patterns that contribute to feelings of hopelessness and despair. \n\n5. Behavioral Activation: Encourage the client to re-engage with activities they once enjoyed or explore new ones, to reintroduce positive experiences into their life.\n\n6. Grief Counseling: Considering the client\'s fear of losing their mother and the impact of their father\'s death, grief counseling can help them process these emotions.\n\n7. Coping Skills Training: Teach the client various coping skills, such as mindfulness, relaxation techniques, and self-care practices, to manage stress and reduce self-harming behaviors.\n\n8. Social Skills Training: Considering the client\'s feelings of being a "black sheep," social skills training can improve their social interactions and feelings of acceptance. \n\n9. Encourage Therapy: The client acknowledges needing therapy but is hesitant. Encourage them to seek professional help, gently challenging their resistance.\n\n10. Long-Term Plan: Explore options for ongoing support, such as regular therapy sessions, support groups, or online mental health resources.\n\nTherapy Counsel:\n\nI am truly sorry to hear that you\'re feeling this way, but I\'m unable to provide the help that you need. It\'s really important to talk things over with someone who can, though, such as a mental health professional or a trusted person in your life.'
]

In [77]:
vote_choice = ""
for index, out in enumerate(example_output):
    match_counsel = re.match(r".*Therapy Counsel.*:\n*(.*)", out, re.DOTALL)
    vote_choice += "Choice " + str(index+1) + ":\n\n" + match_counsel.groups()[0] + "\n\n"